In [56]:
import requests
import json
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
from config import api_key
from pandas_profiling import ProfileReport

pd.options.display.float_format = "{:,.2f}".format

blockbusters_df = pd.read_csv("data/blockbusters.csv")
moviebudget_df= pd.read_csv("data/Movie_budget.csv")
movieid_df= pd.read_csv("data/metadata.csv")

#Merging and cleaning data 
#Merge 1
budgetbuster_df = pd.merge(moviebudget_df,blockbusters_df, how = "right", on = "Title")
budgetbuster_df = budgetbuster_df.drop(columns=["Release Date","length", "rank_in_year", "rating","Year_x", "Worldwide_gross"])
budgetbuster_df = budgetbuster_df.rename(columns={"Year_y":"Year"})
budgetbuster_df = budgetbuster_df.dropna(axis=0,inplace= False)
budgetbuster_df["Production Budget"] = (budgetbuster_df["Production Budget"].astype(float))/1000000
budgetbuster_df["Domestic Gross"] = (budgetbuster_df["Domestic Gross"].astype(float))/1000000
budgetbuster_df["Worldwide Gross"] = (budgetbuster_df["Worldwide Gross"].astype(float))/1000000
budgetbuster_df = budgetbuster_df.drop_duplicates(subset=["Title","Year"],inplace= False)
budgetbuster_df.to_csv("datacreatedcsv/budgetbuster.csv", index_label="Movie ID")
budgetbuster_df

C:\Users\a_don\AppData\Local\Temp\ipykernel_13612\2493813755.py:16: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movieid_df= pd.read_csv("data/metadata.csv")


,Rank,Title,Production Budget,Domestic Gross,Worldwide Gross,Main Genre,2nd Genre,3rd Genre,IMDB Rating,Studio,Year
0,48,Black Panther,200.00,700.06,"1,336.49",Action,Adventure,Drama,7.40,Walt Disney Pictures,2018
1,5,Avengers: Infinity War,300.00,678.82,"2,048.36",Action,Adventure,Sci-Fi,8.50,Walt Disney Pictures,2018
2,50,Incredibles 2,200.00,608.58,"1,242.81",Animation,Action,Adventure,7.80,Pixar,2018
3,144,Jurassic World: Fallen Kingdom,170.00,417.72,"1,308.32",Action,Adventure,Drama,6.20,Universal Pictures,2018
7,11,Solo: A Star Wars Story,275.00,213.77,393.15,Action,Adventure,Drama,7.00,Walt Disney Pictures,2018
...,...,...,...,...,...,...,...,...,...,...,...
442,"4,918",Annie Hall,4.00,38.25,38.25,Romance,Drama,Comedy,8.10,Metro-Goldwyn-Mayer,1977
447,"2,387",The Omen,25.00,54.61,119.30,Thriller,Horror,Drama,7.60,20th Century Fox,1976
450,44,King Kong,207.00,218.08,550.52,Horror,Fantasy,Adventure,5.80,Paramount Pictures,1976
455,445,Midway,100.00,56.85,126.82,History,Drama,Action,6.70,Universal Pictures,1976


In [48]:
#Cleaning movie_id 
movieid_df = movieid_df.drop(["adult","belongs_to_collection","budget","homepage","id","Title","original_language","overview","revenue","runtime",
                                "spoken_languages","status", "tagline","video","vote_average","vote_count","popularity","poster_path","production_companies",
                                "production_countries","genres"], axis=1, inplace = False)
movieid_df = movieid_df.dropna(axis=0,inplace= False)
movieid_df['release_date'] = pd.to_datetime(movieid_df['release_date'])
movieid_df['Year'] = movieid_df['release_date'].dt.year
movieid_df = movieid_df.drop(["release_date"], axis=1, inplace = False)
movieid_df = movieid_df.rename(columns={"imdb_id":"IMDb ID", "title":"Title"})
movieid_df.to_csv("datacreatedcsv/movieid.csv", index_label="Movie ID")
movieid_df

,IMDb ID,Title,Year
0,tt0114709,Toy Story,1995
1,tt0113497,Jumanji,1995
2,tt0113228,Grumpier Old Men,1995
3,tt0114885,Waiting to Exhale,1995
4,tt0113041,Father of the Bride Part II,1995
...,...,...,...
45460,tt0102797,Robin Hood,1991
45462,tt2028550,Century of Birthing,2011
45463,tt0303758,Betrayal,2003
45464,tt0008536,Satan Triumphant,1917


In [49]:
#Merging all csv together
csvmovies_df = pd.merge(budgetbuster_df,movieid_df, how= "left", on=["Title","Year"])
csvmovies_df = csvmovies_df.drop_duplicates(subset=["Title","Year"],inplace= False)
csvmovies_df.to_csv("datacreatedcsv/csvmovies.csv", index_label="Movie ID")
csvmovies_df

,Rank,Title,Production Budget,Domestic Gross,Worldwide Gross,Main Genre,2nd Genre,3rd Genre,IMDB Rating,Studio,Year,IMDb ID
0,48,Black Panther,200.00,700.06,"1,336.49",Action,Adventure,Drama,7.40,Walt Disney Pictures,2018,NaN
1,5,Avengers: Infinity War,300.00,678.82,"2,048.36",Action,Adventure,Sci-Fi,8.50,Walt Disney Pictures,2018,NaN
2,50,Incredibles 2,200.00,608.58,"1,242.81",Animation,Action,Adventure,7.80,Pixar,2018,NaN
3,144,Jurassic World: Fallen Kingdom,170.00,417.72,"1,308.32",Action,Adventure,Drama,6.20,Universal Pictures,2018,NaN
4,11,Solo: A Star Wars Story,275.00,213.77,393.15,Action,Adventure,Drama,7.00,Walt Disney Pictures,2018,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
217,"4,918",Annie Hall,4.00,38.25,38.25,Romance,Drama,Comedy,8.10,Metro-Goldwyn-Mayer,1977,tt0075686
218,"2,387",The Omen,25.00,54.61,119.30,Thriller,Horror,Drama,7.60,20th Century Fox,1976,tt0075005
219,44,King Kong,207.00,218.08,550.52,Horror,Fantasy,Adventure,5.80,Paramount Pictures,1976,tt0074751
220,445,Midway,100.00,56.85,126.82,History,Drama,Action,6.70,Universal Pictures,1976,tt0074899


In [42]:
#Retrieving Data from OMDb 
#Adding director based on title with null IMDb ID
NaNid_df = csvmovies_df[csvmovies_df["IMDb ID"].isnull()]
title =NaNid_df["Title"].tolist()
url = "http://www.omdbapi.com/?apikey=" + api_key + "&t="
title_response = []

for movie in title:
    movie_data = requests.get(url + movie).json()
    
    try:
        directors = movie_data["Director"]
    except:
        diretors = "N/A"


    title_response.append({ "Title":movie,
                    "Director": directors})
  
    print(f"{movie} was directed by {directors}")



Black Panther was directed by Ryan Coogler
Avengers: Infinity War was directed by Anthony Russo, Joe Russo
Incredibles 2 was directed by Brad Bird
Jurassic World: Fallen Kingdom was directed by J.A. Bayona
Solo: A Star Wars Story was directed by Ron Howard
Venom was directed by Ruben Fleischer
Jumanji: Welcome to the Jungle was directed by Jake Kasdan
Spider-Man: Homecoming was directed by Jon Watts
It was directed by Andy Muschietti
WALL-E was directed by Andrew Stanton
300 was directed by Zack Snyder
Superman II was directed by Richard Lester, Richard Donner


In [43]:
#Retrieving Data from OMDb 
#Adding director based on title with null IMDb ID
imdbid_df = csvmovies_df.dropna(axis=0,inplace= False)
imdbid = imdbid_df["IMDb ID"].tolist()
url = "http://www.omdbapi.com/?apikey=" + api_key + "&i="
id_response = []

for id in imdbid:
    movie_data = requests.get(url + id).json()
    
    try:
        directors = movie_data["Director"]
    except:
        diretors = "N/A"


    id_response.append({ "IMDb ID":id,
                    "Director": directors})
  
    print(f"{id} was directed by {directors}")

tt0451279 was directed by Patty Jenkins
tt3501632 was directed by Taika Waititi
tt0974015 was directed by Zack Snyder
tt3748528 was directed by Gareth Edwards
tt3498820 was directed by Anthony Russo, Joe Russo
tt1431045 was directed by Tim Miller
tt2948356 was directed by Byron Howard, Rich Moore, Jared Bush
tt2975590 was directed by Zack Snyder
tt3470600 was directed by Garth Jennings, Christophe Lourdelet
tt0369610 was directed by Colin Trevorrow
tt2395427 was directed by Joss Whedon
tt2820852 was directed by James Wan
tt2293640 was directed by Kyle Balda, Pierre Coffin
tt1951266 was directed by Francis Lawrence
tt2109248 was directed by Michael Bay
tt2015381 was directed by James Gunn
tt1587310 was directed by Robert Stromberg
tt1877832 was directed by Bryan Singer
tt1843866 was directed by Anthony Russo, Joe Russo
tt1872181 was directed by Marc Webb
tt2103281 was directed by Matt Reeves
tt2294629 was directed by Chris Buck, Jennifer Lee
tt1300854 was directed by Shane Black
tt16909

In [50]:
NaNdirect_df = pd.DataFrame(title_response)
IDdirect_df = pd.DataFrame(id_response)

In [51]:
NaNmovies_df = pd.merge(csvmovies_df,NaNdirect_df, on = "Title")
NaNmovies_df.to_csv("datacreatedcsv/NaNmovies.csv", index_label="Movie ID")
NaNmovies_df

,Rank,Title,Production Budget,Domestic Gross,Worldwide Gross,Main Genre,2nd Genre,3rd Genre,IMDB Rating,Studio,Year,IMDb ID,Director
0,48,Black Panther,200.00,700.06,"1,336.49",Action,Adventure,Drama,7.40,Walt Disney Pictures,2018,NaN,Ryan Coogler
1,5,Avengers: Infinity War,300.00,678.82,"2,048.36",Action,Adventure,Sci-Fi,8.50,Walt Disney Pictures,2018,NaN,"Anthony Russo, Joe Russo"
2,50,Incredibles 2,200.00,608.58,"1,242.81",Animation,Action,Adventure,7.80,Pixar,2018,NaN,Brad Bird
3,144,Jurassic World: Fallen Kingdom,170.00,417.72,"1,308.32",Action,Adventure,Drama,6.20,Universal Pictures,2018,NaN,J.A. Bayona
4,11,Solo: A Star Wars Story,275.00,213.77,393.15,Action,Adventure,Drama,7.00,Walt Disney Pictures,2018,NaN,Ron Howard
5,353,Venom,116.00,213.51,856.08,Action,Adventure,Drama,6.90,Columbia Pictures,2018,NaN,Ruben Fleischer
6,503,Jumanji: Welcome to the Jungle,90.00,404.51,961.63,Action,Adventure,Comedy,7.00,Sony Pictures,2017,NaN,Jake Kasdan
7,127,Spider-Man: Homecoming,175.00,334.20,878.27,Action,Adventure,Drama,7.50,Sony Pictures,2017,NaN,Jon Watts
8,1785,It,35.00,328.83,701.01,Horror,Drama,Mystery,7.40,Warner Bros,2017,NaN,Andy Muschietti
9,114,WALL-E,180.00,223.81,532.51,Animation,Animation,Adventure,8.40,Pixar,2008,NaN,Andrew Stanton


In [52]:
IDmovies_df = pd.merge(csvmovies_df,IDdirect_df, on = "IMDb ID")
IDmovies_df.to_csv("datacreatedcsv/IDmovies.csv", index_label="Movie ID")
IDmovies_df

,Rank,Title,Production Budget,Domestic Gross,Worldwide Gross,Main Genre,2nd Genre,3rd Genre,IMDB Rating,Studio,Year,IMDb ID,Director
0,192,Wonder Woman,150.00,412.56,817.69,Action,Adventure,Drama,7.50,Warner Bros,2017,tt0451279,Patty Jenkins
1,110,Thor: Ragnarok,180.00,315.06,850.48,Action,Adventure,Drama,7.90,Walt Disney Pictures,2017,tt3501632,Taika Waititi
2,7,Justice League,300.00,229.02,655.95,Action,Adventure,Drama,6.50,Warner Bros,2017,tt0974015,Zack Snyder
3,51,Rogue One: A Star Wars Story,200.00,533.54,"1,055.08",Action,Adventure,Drama,7.80,Walt Disney Pictures,2016,tt3748528,Gareth Edwards
4,18,Captain America: Civil War,250.00,408.08,"1,151.90",Action,Adventure,Sci-Fi,7.80,Walt Disney Pictures,2016,tt3498820,"Anthony Russo, Joe Russo"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,"4,918",Annie Hall,4.00,38.25,38.25,Romance,Drama,Comedy,8.10,Metro-Goldwyn-Mayer,1977,tt0075686,Woody Allen
205,"2,387",The Omen,25.00,54.61,119.30,Thriller,Horror,Drama,7.60,20th Century Fox,1976,tt0075005,Richard Donner
206,44,King Kong,207.00,218.08,550.52,Horror,Fantasy,Adventure,5.80,Paramount Pictures,1976,tt0074751,John Guillermin
207,445,Midway,100.00,56.85,126.82,History,Drama,Action,6.70,Universal Pictures,1976,tt0074899,Jack Smight


In [53]:
allmovies_df = NaNmovies_df.append(IDmovies_df)
allmovies_df.to_csv("allmoviesdata.csv", index_label="Movie ID")

C:\Users\a_don\AppData\Local\Temp\ipykernel_13612\707215395.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  allmovies_df = NaNmovies_df.append(IDmovies_df)


In [58]:
profile = ProfileReport(allmovies_df)
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Changes in Cost Over the Years

* main - line

* 1990's

* 2000's

* 2010's

Production Costs per Genre
* pie chart - broken by genre
* pie chart - production companies

Production Cost vs Domestic and Worldwide Gross
* production cost vs domestic - scatter plot
* production cost vs worldwide

Production Cost vs Viewer Ratings
* scatter plot

Director
* bar - combined gross for movies

* top 100 movies directors